In [1]:
import sys
import os
import requests
import datetime
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))

import time
import talib
import sqlite3 as sqlite3
import yfinance as yf
import pandas as pd
from util.util import get_ucodes

path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')

In [2]:
conn = sqlite3.connect(path_sqlite)
cursor = conn.cursor()

ucodes = get_ucodes()
data = {}
for ucode in ['2800.HK', '2828.HK', '2823.HK', 'spy', 'vti', 'qqq', 'shy', 'tlt']: # '3188.HK', 'voo', 'tqqq'
    if '.HK' in ucode:
        ucode = ucode.replace('.HK', '').zfill(5)
    
    #
    sql = """SELECT t.code, t.lot, t.nmll, t.stime, t.high, t.low, t.open, t.close, t.volume
                FROM (SELECT n.code, n.lot, n.nmll, c.stime, c.high, c.low, c.open, c.close, c.volume 
                    FROM s_{} AS c INNER JOIN name AS n 
                        ON c.code=n.code ORDER BY c.stime DESC) AS t 
                            ORDER BY t.stime """.format(ucode)
    cursor.execute(sql)
    columns = ['code', 'lot', 'nmll', 'sdate', 'high', 'low', 'open', 'last', 'vol']
    data[ucode] = pd.DataFrame(cursor.fetchall(), columns=columns)
    data[ucode].index = pd.to_datetime(data[ucode].sdate)
conn.close()

In [3]:
data_all = []
nmll = []
for k, df in data.items():
    data_all.append(df['last'])
    nmll.append(k)
df2 = pd.concat(data_all, axis=1)
df2 = df2.dropna()
df2.columns = nmll
print(f"{df2.index[0]} ~ {df2.index[-1]}  {df2.shape}")

2008-01-10 00:00:00 ~ 2021-07-29 00:00:00  (3248, 8)


In [4]:
# 相關系數
df2.corr()

,02800,02828,02823,spy,vti,qqq,shy,tlt
02800,1.000000,0.518135,0.585320,0.759127,0.761884,0.691396,0.578264,0.555182
02828,0.518135,1.000000,0.534238,-0.038912,-0.034001,-0.052840,-0.164512,-0.239070
02823,0.585320,0.534238,1.000000,0.535331,0.534034,0.591606,0.345789,0.320148
spy,0.759127,-0.038912,0.535331,1.000000,0.999695,0.983925,0.903856,0.891760
vti,0.761884,-0.034001,0.534034,0.999695,1.000000,0.983509,0.902819,0.887064
qqq,0.691396,-0.052840,0.591606,0.983925,0.983509,1.000000,0.911645,0.884587
shy,0.578264,-0.164512,0.345789,0.903856,0.902819,0.911645,1.000000,0.948663
tlt,0.555182,-0.239070,0.320148,0.891760,0.887064,0.884587,0.948663,1.000000


In [5]:
# 收益率% 年度
df3_1 = df2.copy(deep=True)
df3 = pd.DataFrame(columns=['Date']+df3_1.columns.to_list())
for year in range(2008, 2022, 1):
    df3_2 = df3_1[f'{year}-01-01': f'{year}-12-31'].iloc[[0, -1]].pct_change()*100
    df3.loc[df3.shape[0]+1] = [f'{year}-01-01 ~ {year}-12-31']+df3_2.iloc[-1].to_list()
print(df3.std())
df3

02800    22.532206
02828    24.351570
02823    36.552512
spy      16.818742
vti      17.204325
qqq      22.401136
shy       1.685974
tlt      16.204904
dtype: float64


,Date,02800,02828,02823,spy,vti,qqq,shy,tlt
1,2008-01-01 ~ 2008-12-31,-46.812386,-50.787916,-68.571429,-34.594076,-34.367963,-37.805079,6.067687,33.335582
2,2009-01-01 ~ 2009-12-31,42.951252,51.263537,72.261072,23.843364,26.536814,49.614488,1.010508,-19.406402
3,2010-01-01 ~ 2010-12-31,5.466970,-0.862068,-13.443073,13.110404,15.555596,18.844607,2.022522,7.915778
4,2011-01-01 ~ 2011-12-31,-21.271186,-23.026316,-19.969040,0.852376,-0.062681,1.885604,1.464611,35.019615
5,2012-01-01 ~ 2012-12-31,20.399579,11.241446,4.356061,12.262804,12.789261,13.489277,0.309565,5.797544
6,2013-01-01 ~ 2013-12-31,-0.212766,-8.578641,-19.759450,29.001458,30.146335,32.425415,0.238341,-12.207678
7,2014-01-01 ~ 2014-12-31,1.702128,12.837212,39.720129,14.561700,13.543720,20.123711,0.399435,26.915664
8,2015-01-01 ~ 2015-12-31,-8.108108,-21.231770,-19.970414,1.288512,0.430834,9.765484,0.322856,-2.869509
9,2016-01-01 ~ 2016-12-31,3.030303,0.425534,3.944773,13.585741,14.530758,9.406113,0.691310,0.446896
10,2017-01-01 ~ 2017-12-31,34.753363,23.964341,40.636704,20.781433,20.295473,31.487210,0.345907,8.717187


In [6]:
# 收益率% 半年度
df3_1 = df2.copy(deep=True)
df3 = pd.DataFrame(columns=['Date']+df3_1.columns.to_list())
for year in range(2008, 2022, 1):
    df3_2 = df3_1[f'{year}-01-01': f'{year}-06-30'].iloc[[0, -1]].pct_change()*100
    df3.loc[df3.shape[0]+1] = [f'{year}-01-01 ~ {year}-06-30']+df3_2.iloc[-1].to_list()
    
    df3_3 = df3_1[f'{year}-07-01': f'{year}-12-31'].iloc[[0, -1]].pct_change()*100
    df3.loc[df3.shape[0]+1] = [f'{year}-07-01 ~ {year}-12-31']+df3_3.iloc[-1].to_list()
print(df3.std())
df3

02800    13.673241
02828    15.381758
02823    23.876028
spy      11.347206
vti      11.794125
qqq      13.316133
shy       1.099031
tlt      11.990441
dtype: float64


,Date,02800,02828,02823,spy,vti,qqq,shy,tlt
1,2008-01-01 ~ 2008-06-30,-18.214936,-25.611960,-45.864662,-8.516597,-7.155083,-5.736978,1.468579,0.651441
2,2008-07-01 ~ 2008-12-31,-33.786848,-32.133651,-38.257016,-27.485247,-28.166813,-33.340955,4.546482,32.096384
3,2009-01-01 ~ 2009-06-30,23.583663,32.490975,61.538462,0.184201,1.723337,17.579630,0.244750,-17.518970
4,2009-07-01 ~ 2009-12-31,17.551463,14.168936,3.646564,26.561185,27.000642,30.033142,0.557024,-2.322548
5,2010-01-01 ~ 2010-06-30,-6.378132,-9.404389,-21.262003,-8.110660,-7.479456,-7.537968,1.735304,15.268879
6,2010-07-01 ~ 2010-12-31,14.320988,12.145387,9.739130,24.322698,26.048309,29.259754,0.314623,-5.775365
7,2011-01-01 ~ 2011-06-30,-3.177966,-2.089781,0.154799,4.840910,5.150450,3.523118,0.797620,2.655480
8,2011-07-01 ~ 2011-12-31,-20.086022,-23.085851,-21.428571,-5.127176,-6.286756,-3.526486,0.576923,31.442917
9,2012-01-01 ~ 2012-06-30,4.942166,-5.229722,-4.356061,7.768399,7.888539,13.210291,0.085357,6.068917
10,2012-07-01 ~ 2012-12-31,13.086420,15.650408,8.893281,3.177317,3.414805,-0.881297,0.216924,-0.509890
